In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import glob
import os

# ===============================
# ▶ 読み込み元データの保存場所と予測モデルの保存場所
# ▶ 予測モデルが更新されたら、04_学習環境フォルダに移動させること（2025-07-26）
# ===============================
DATA_DIR = r"E:\\01_データセット\\特徴量\\"
MODEL_DIR = r"E:\\03_予測モデル\\"

# ===============================
# ▶ データ読み込み（元データの数をコントロールするために分割して保存している）
# ▶ globでファイルを取得している。複数ある場合はpd.concatで合体している
# ===============================
file_list = glob.glob(os.path.join(DATA_DIR, '特徴量*.csv'))
df_list = [pd.read_csv(file, encoding='shift_jis') for file in file_list]
df_all = pd.concat(df_list, ignore_index=True)

#================================
# race_id ユニーク化（1レースごとの予測をするためにグループ化している）
#================================
unique_race_ids = np.arange(1, len(df_all) // 6 + 1).repeat(6)
df_all['race_id'] = unique_race_ids

# ===============================
# ▶ 特徴量 & 目的変数
# ===============================
# 調整箇所じー(´◉ω◉` )
feature_cols = ['motor_win_rate', 'course', 'Zenkoku_power', 'Boat_power']
target_col = 'result'

# ===============================
# 特徴量の調整（コースは小さいほうが強い。特徴量に－をかけて反転している）-->（2025-07-26）－にしないほうが的中率が高かった
# ===============================
# 調整箇所じー(´◉ω◉` )
#df_all['course'] = df_all['course'] * -1

# ===============================
# 正規化
# ===============================
scaler = StandardScaler()
X = scaler.fit_transform(df_all[feature_cols])
y = df_all[target_col].values - 1  # 0始まりに変換

# ===============================
# ▶ Dataset 作成
# ▶ pytorchのカスタムデータをセットしている。機械学習モデルに渡す特徴量と正解データを定型的な形で提供するための仕組み
# ===============================
class BoatRaceDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float32),
            'targets': torch.tensor(self.targets[idx], dtype=torch.long)
        }
# ===============================
# データ分割 test_size = 0.3 random_state = 0 --> 2025/07/20 -->0.2にしたほうが的中率があがった -->0.4 -->0.3
# ===============================
from sklearn.model_selection import train_test_split
# 調整箇所じー(´◉ω◉` )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

train_dataset = BoatRaceDataset(X_train, y_train)
test_dataset = BoatRaceDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ===============================
# ▶ モデル imput_dim XXX XXX, YYY, YYY ZZZ
# ===============================
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
# 調整箇所じー(´◉ω◉` )　下と連動すること→変更したらBoatPy_verXX.pyも連動して変えること。エラーになります。
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 48)
        self.fc4 = nn.Linear(48, 32)
        self.fc5 = nn.Linear(32, 24)                
        self.fc6 = nn.Linear(24, 16)
        self.fc7 = nn.Linear(16, 8)
        self.output = nn.Linear(8, num_classes)

    def forward(self, x):
# 調整箇所じー(´◉ω◉` )　上と連動すること
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x)) 
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))          
        x = self.output(x)
        return x

# ===============================
# ▶ 訓練　pcにgpuがあるかをチェックしている。gpuがあればcuda、なければcpu
# ▶ 自分のニューラルネットワーククラスを用いて、モデルを作成している。出力クラスを6つ（6艇が出走するから）にしている分類問題
# ===============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ClassificationModel(input_dim=X.shape[1], num_classes=6).to(device)
# ===============================
# ▶ 学習率 lr= 0.001 --> 2025/07/25
# ▶ 学習率 lr=
# ===============================
# 調整箇所じー、学習率(´◉ω◉` )　--> 学習率を0.001から0.0005にしたほうが的中率があがった
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

# ===============================
# 調整箇所じー、エポック(´◉ω◉` )　for epoch in range(50) -->2025/07/25
# エポックとは --> 50回繰り返しする --> 30にする
# ===============================
for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in train_loader:
        features = batch['features'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

# ===============================
# ▶ モデル保存 --> 初めに指定したMODEL_DIRに作成したモデルを保存している -->実際に使うときは04_学習環境に移動させること
# ===============================
os.makedirs(MODEL_DIR, exist_ok=True)
model_path = os.path.join(MODEL_DIR, 'classification_model.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler
}, model_path)

print(f'Model saved to {model_path}')

# ===============================
# ▶ 予測 --> モデルの性能を試すために、順位を除いた予測用.csvに適用させる。的中率はエクセルでのちに計算する。
# ===============================
predict_file = os.path.join(DATA_DIR, '予測用.csv')
df_predict = pd.read_csv(predict_file, encoding='shift_jis')

X_predict = scaler.transform(df_predict[feature_cols])

model.eval()
with torch.no_grad():
    X_tensor = torch.tensor(X_predict, dtype=torch.float32).to(device)
    outputs = model(X_tensor)
    predictions = torch.argmax(outputs, dim=1).cpu().numpy() + 1  # 1始まりに戻す

df_predict['result'] = predictions

# ===============================
# ▶ 順位強制付与（同点の場合はcourseが小さい方を優先 -->内側有利の原則にのっとって）　　
# ===============================

def rank_with_tiebreaker(df):
    results = []
    for race_id, group in df.groupby('race_id'):
        group = group.copy()
        group['rank_score'] = group['result'] + group['course'] * 0.01  # courseで微調整
        group['final_rank'] = group['rank_score'].rank(method='first', ascending=True).astype(int)
        results.append(group)
    return pd.concat(results)

df_predict = rank_with_tiebreaker(df_predict)
df_predict['result'] = df_predict['final_rank']

# ===============================
# ▶ 予測用.csvに結果をつけて保存 --> エクセルで分析へ
# ===============================

predict_result_file = os.path.join(DATA_DIR, '予測用_結果付きB.csv')
df_predict.to_csv(predict_result_file, index=False, encoding='utf-8')
print(f'Prediction results saved to {predict_result_file}')

Epoch 1, Loss: 1.8150
Epoch 2, Loss: 1.8081
Epoch 3, Loss: 1.8044
Epoch 4, Loss: 1.8011
Epoch 5, Loss: 1.7962
Epoch 6, Loss: 1.7855
Epoch 7, Loss: 1.7765
Epoch 8, Loss: 1.7618
Epoch 9, Loss: 1.7469
Epoch 10, Loss: 1.7344
Epoch 11, Loss: 1.7146
Epoch 12, Loss: 1.6969
Epoch 13, Loss: 1.6848
Epoch 14, Loss: 1.6593
Epoch 15, Loss: 1.6519
Epoch 16, Loss: 1.6357
Epoch 17, Loss: 1.6322
Epoch 18, Loss: 1.6258
Epoch 19, Loss: 1.6251
Epoch 20, Loss: 1.6166
Epoch 21, Loss: 1.6175
Epoch 22, Loss: 1.6171
Epoch 23, Loss: 1.6137
Epoch 24, Loss: 1.6123
Epoch 25, Loss: 1.6139
Epoch 26, Loss: 1.6081
Epoch 27, Loss: 1.6175
Epoch 28, Loss: 1.6099
Epoch 29, Loss: 1.6098
Epoch 30, Loss: 1.6095
Epoch 31, Loss: 1.5954
Epoch 32, Loss: 1.5914
Epoch 33, Loss: 1.6037
Epoch 34, Loss: 1.6053
Epoch 35, Loss: 1.5993
Epoch 36, Loss: 1.6158
Epoch 37, Loss: 1.5943
Epoch 38, Loss: 1.5869
Epoch 39, Loss: 1.5984
Epoch 40, Loss: 1.5916
Epoch 41, Loss: 1.5862
Epoch 42, Loss: 1.5920
Epoch 43, Loss: 1.5947
Epoch 44, Loss: 1.59